In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np 
import math

def pvd_cons(x):
    """
    :return: All cons should be minus than 0
    """
    con1 = -x[0] + 0.0193 * x[2]
    con2 = -x[1] + 0.00954 * x[2]
    con3 = -np.pi * x[2] ** 2 * x[3] - 4 / 3 * np.pi * x[2] ** 3 + 1296000
    con4 = x[3] - 240
    return [con1, con2, con3, con4]


def pvd_obj(x):
    """
    :param x:
    0 <= X[0] <= 99,
    0 <= X[1] <= 99,
    10 <= X[2] <= 200,
    10 <= X[3] <= 200
    :return:
    """
    return 0.6224 * x[0] * x[2] * x[3] + 1.7781 * x[1] * x[2] ** 2 + 3.1661 * x[0] ** 2 * x[3] + 19.84 * x[0] ** 2 * x[2]

def penalty_fuc(x, cons):
    pen_temp = 0
    for i in range(len(cons)):
        if cons[i] > 0:
            pen_temp += 1e6 * cons[i] ** 2
    return pen_temp


def fitness_function(x):
    # Calculate the objective value
    obj_value = pvd_obj(x)
    
    # Calculate constraint violations
    constraints = pvd_cons(x)
    penalty = penalty_fuc(x , constraints) # Penalty for violating constraints
    
    # Return the penalized objective value
    return obj_value + penalty  # 1e6 is a large penalty factor

def initialize_population(P, n, lb, ub):
    population = []
    for _ in range(P):
        # Initialize position (VECTOR)
        pos = lb + np.random.rand(*lb.shape) * (ub - lb)  # Use np.random.rand(n), not np.random.rand()

        # Initialize density and volume (VECTORS)
        dens = np.random.rand(n)
        vol = np.random.rand(n)

        # Initialize acceleration (VECTOR)
        acc = lb + np.random.rand(*lb.shape) * (ub - lb)  # Use np.random.rand(n)
        score = fitness_function(pos)
        particle = Particle(acc, dens, vol, pos, score)
        population.append(particle)
    return population

def update_densities_vol(x ,Instance):
    #x is a particle
    x.dens = x.dens+np.random.uniform()*(Instance.g_best.dens - x.dens)
    x.vol = x.vol+np.random.uniform()*(Instance.g_best.vol - x.vol)

def transfer_factor(IterCurr , IterMax):
    return math.exp((IterCurr-IterMax)/IterMax)

def density_factor(IterCurr, IterMax):
    return math.exp((IterMax - IterCurr) / IterMax - (IterCurr / IterMax))

def calc_best(Population):
    best_particle = Population[0]
    best_particle_score = best_particle.score
    for _ in Population:
        if _.score < best_particle_score:
            best_particle = _
            best_particle_score = best_particle.score
    return best_particle_score, best_particle


def calc_acc_exploration(x, Instance):
    P = len(Instance.Population)
    random_material = Instance.Population[np.random.randint(0, P)]
    denmr = random_material.dens
    accmr = random_material.acc
    volmr = random_material.vol

    epsilon = 1e-8  
    x.dens = np.maximum(x.dens, epsilon)  
    x.vol = np.maximum(x.vol, epsilon)  

    x.acc = (denmr + volmr * accmr) / (x.dens * x.vol + epsilon)
    x.acc = np.clip(x.acc, -1e8, 1e8)


def calc_acc_exploitation(x, Instance):
    denbest = Instance.g_best.dens
    accbest = Instance.g_best.acc
    volbest = Instance.g_best.vol

    epsilon = 1e-8  
    x.dens = np.maximum(x.dens, epsilon)
    x.vol = np.maximum(x.vol, epsilon)

    x.acc = (denbest + volbest * accbest) / (x.dens * x.vol + epsilon)
    x.acc = np.clip(x.acc, -1e8, 1e8)


def norm_acc(x, u, l, Instance):
    accmin = min(np.min(p.acc) for p in Instance.Population)  # Use np.min for arrays
    accmax = max(np.max(p.acc) for p in Instance.Population)  # Use np.max for arrays
    if accmax == accmin:  # Handle edge case
        return u * 0 + l
    return u * (x.acc - accmin) / (accmax - accmin) + l

def position_update_exploration(x, IterCurr, IterMax, Instance):
    acc_norm = norm_acc(x, 0.9, 0.1, Instance)
    d = density_factor(IterCurr, IterMax)
    P = len(Instance.Population)
    x_rand = Instance.Population[np.random.randint(0, P)]
    x.pos = x.pos + Instance.C1 * np.random.rand(Instance.n) * acc_norm * d * (x_rand.pos - x.pos)
    x.pos = np.clip(x.pos, Instance.lb, Instance.ub)  # Works if lb/ub are arrays
    x.score = fitness_function(x.pos)


def position_update_exploitation(x, IterCurr, IterMax, Instance):
    x_best = Instance.g_best
    aux_P = 2 * np.random.uniform() - Instance.C4
    d = density_factor(IterCurr, IterMax)
    acc_norm = norm_acc(x, 0.9, 0.1, Instance)
    T = Instance.C3 * transfer_factor(IterCurr, IterMax)
    F = -1 if aux_P > 0.5 else 1
    x.pos = x_best.pos + F * Instance.C2 * np.random.rand(Instance.n) * acc_norm * d * (T * x_best.pos - x.pos)
    x.pos = np.clip(x.pos, Instance.lb, Instance.ub)  # Works if lb/ub are arrays
    x.score = fitness_function(x.pos)

def position_update_migration(x, Instance):
    n = Instance.n
    # Use df=n and size=n
    x.pos = x.pos + np.random.standard_t(df=n, size=n) * (Instance.g_best.pos - x.pos)
    update_densities_vol(x, Instance)

def position_update_foraging(x, Instance):
    n = Instance.n
    random1v1 = np.random.choice([-1, 1], size=n)  # Use NumPy array
    random1v2 = np.random.choice([-1, 1], size=n)  # Use NumPy array
    x.pos = (x.pos + random1v1 * Instance.g_best.pos + 
             np.random.normal(0, 1, size=n) * abs(np.random.normal(0, 1, size=n) * Instance.g_best.pos + 
             random1v2 * x.pos)) / np.random.chisquare(n)

def  calc_diveristy(Instance):
    P = Instance.P
    distance = 0
    for i in range(P):
        for j in range (i+1 , P):
            distance += np.linalg.norm(Instance.Population[i].pos - Instance.Population[j].pos)
    num_pairs = P*(P-1)/2
    return (distance/num_pairs)        


class Particle: 
    def __init__(self , acc , dens , vol , pos , score):
        self.vol = vol
        self.dens = dens
        self.pos = pos
        self.acc = acc
        self.score = score

class AOA_FSA:
    def __init__(self, P, n, lb, ub):
        self.n = n
        self.P = P
        self.Population = initialize_population(P, n, lb, ub)
        self.g_best_score, self.g_best = calc_best(self.Population)
        self.C1 = 2
        self.C2 = 6
        self.C3 = 2
        self.C4 = 0.5
        self.lb = lb
        self.ub = ub
        self.initial_diversity  = calc_diveristy(self)
        self.MPb = 0.1

    def setBest(self):
        self.g_best_score, self.g_best = calc_best(self.Population)
        

if __name__ == "__main__":
    min_score = math.inf
    for super in range(5):
        print("super" , super+1)
        P = 50
        IterMax = 1000
        lb = np.array([0, 0, 10 , 10])
        ub = np.array([99, 99, 200 , 200])
        n =  len(lb)
        Instance = AOA_FSA(P , n , lb , ub)
        for i in range(1 , IterMax+1):
            #at every some iteration , we are gonna calculate diversity
            #according to that diversity we will decide whether to use flamingo search or not
            #migration behaviour will be only applied to bottom low fitness individuals
            #foraging will also be applied on middle individuals for efficient exploitation at this stage according to flamingo rule
            new_transfer_factor = transfer_factor(i , IterMax)
            flamingo_flag = False
            migrating_population = int(P * Instance.MPb)  # Convert to integer
            foraging_population = int(P * np.random.rand() * 0.7 * (1 - Instance.MPb))  # Convert to integer
            if((i%5)== 0):
                curr_diversity = calc_diveristy(Instance)
                if(curr_diversity < 0.7*Instance.initial_diversity and new_transfer_factor<=0.5):
                    flamingo_flag =True 
                    Instance.Population.sort(key=lambda x:x.score , reverse=True)
            if(flamingo_flag):
                for k in range(migrating_population):
                    position_update_migration(Instance.Population[k], Instance)
                for k in range(migrating_population , migrating_population+foraging_population):
                    position_update_foraging(Instance.Population[k] , Instance)
                for k in range(migrating_population+foraging_population , P):
                    update_densities_vol(Instance.Population[k] , Instance)
                    position_update_exploration(Instance.Population[k] , i , IterMax , Instance)
            else:
                for obj in Instance.Population:
                    update_densities_vol(obj , Instance)
                    if(new_transfer_factor<= 0.5):
                        calc_acc_exploration(obj , Instance)
                        position_update_exploration(obj , i , IterMax , Instance)
                    else:
                        calc_acc_exploitation(obj , Instance)
                        position_update_exploitation(obj , i , IterMax , Instance)
            Instance.setBest()
            min_score = min(min_score , Instance.g_best_score)
        print(min_score)
    print("The best fitness score is" , min_score)

super 1
6044.999474684658
super 2
6044.999474684658
super 3
6044.999474684658
super 4
6000.49624598411
super 5
6000.49624598411
The best fitness score is 6000.49624598411
